# Segmentation by registration and feature extraction

In [8]:
import glob
import os
import os.path
import numpy as np
import pandas as pd
import maweight
import pickle
import logging
import tqdm
from tqdm.contrib import tzip

from config import manually_segmented_path, dissected_path, path_prefix_data
from config import output_path, skull_features_path, head_features_path 
from config import save_registered_images
from config import bin_width, bin_min, bin_max, threshold
from config import tmp_path, elastix_params, threads

manually_segmented_bone_path = os.path.join(path_prefix_data, "manually_labels_bone")
dissected_bone_path = os.path.join(path_prefix_data, "dissected_bone")        

bone_registration_path = os.path.join(path_prefix_data, "bone_regsitration")
os.makedirs(bone_registration_path, exist_ok=True)

registration_after_bone_registration = True

image_format = "." + elastix_params['ResultImageFormat']

import warnings
warnings.filterwarnings('ignore')

LIMIT=None

# setting the logging format
FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

In [3]:
#delete all '_' character from filenames
def rename_files(files):
    new_files = [os.path.join(os.path.dirname(file), os.path.basename(file).replace("_", "")) for file in files]
    [os.rename(rfiles[0], rfiles[1]) for rfiles in zip(files, new_files)]
    return new_files

## Discovering files to process

In [4]:
manually_segmented_files = []
manually_segmented_files += sorted(glob.glob(os.path.join(manually_segmented_path, '*.nii')))
manually_segmented_files += sorted(glob.glob(os.path.join(manually_segmented_path, '*.nii.gz')))

manually_segmented_files = rename_files(manually_segmented_files)

manually_segmented_images= [f for f in manually_segmented_files if not 'head' in 
                            f.split(os.sep)[-1] and not 'skull' in f.split(os.sep)[-1]]
manually_segmented_head= [f for f in manually_segmented_files if 'head' in f]
manually_segmented_skulls= [f for f in manually_segmented_files if 'skull' in f]

dissected_images = []
dissected_images = glob.glob(os.path.join(dissected_path, '*.nii'))
dissected_images += glob.glob(os.path.join(dissected_path, '*.nii.gz'))
dissected_images = sorted(dissected_images)

manually_segmented_bones = []
if os.path.exists(manually_segmented_bone_path):
    manually_segmented_bones = glob.glob(os.path.join(manually_segmented_bone_path, '*.nii'))
    manually_segmented_bones += glob.glob(os.path.join(manually_segmented_bone_path, '*.nii.gz'))
    manually_segmented_bones = sorted(manually_segmented_bones)

dissected_bones = []
if os.path.exists(dissected_bone_path):
    dissected_bones = glob.glob(os.path.join(dissected_bone_path, '*.nii'))
    dissected_bones += glob.glob(os.path.join(dissected_bone_path, '*.nii.gz'))
    dissected_bones = sorted(dissected_bones)    


if LIMIT:
    dissected_images= dissected_images[:LIMIT]

dissected_images = rename_files(dissected_images)
print(f"Number of dissected images: {len(dissected_images)}")
print(f"Number of dissected bone masks: {len(dissected_bones)}")
print(f"Number of manually segmented images: {len(manually_segmented_images)}")
print(f"Number of skull masks: {len(manually_segmented_skulls)}")
print(f"Number of head masks: {len(manually_segmented_head)}")
print(f"Number of bone masks: {len(manually_segmented_bones)}")


Number of dissected images: 170
Number of dissected bone masks: 170
Number of manually segmented images: 20
Number of skull masks: 20
Number of head masks: 20
Number of bone masks: 20


## Set Origin to Center of Mass

In [4]:
for d, b in zip(dissected_images, dissected_bones): 
    maweight.origin_to_center_of_mass(d, [b])
for i, h, m, b in zip(manually_segmented_images, manually_segmented_skulls, manually_segmented_head, manually_segmented_bones):
    maweight.origin_to_center_of_mass(i, [h, m, b])

Processing image: data/dissected/197a.nii.gz and masks: []
Center of Mass of data/dissected/197a.nii.gz is already in origin.
Processing image: data/dissected/197f.nii.gz and masks: []
Center of Mass of data/dissected/197f.nii.gz is already in origin.
Processing image: data/dissected/197k.nii.gz and masks: []
Center of Mass of data/dissected/197k.nii.gz is already in origin.
Processing image: data/dissected/198a.nii.gz and masks: []
Center of Mass of data/dissected/198a.nii.gz is already in origin.
Processing image: data/dissected/198f.nii.gz and masks: []
Center of Mass of data/dissected/198f.nii.gz is already in origin.
Processing image: data/dissected/198k.nii.gz and masks: []
Center of Mass of data/dissected/198k.nii.gz is already in origin.
Processing image: data/dissected/199a.nii.gz and masks: []
Center of Mass of data/dissected/199a.nii.gz is already in origin.
Processing image: data/dissected/199k.nii.gz and masks: []
Center of Mass of data/dissected/199k.nii.gz is already in 

## Segmentation by Registration with bone

In [7]:
for d, db in tzip(dissected_images, dissected_bones):
    for (i, h, m, b) in zip(manually_segmented_images, manually_segmented_head, manually_segmented_skulls, manually_segmented_bones):
        output_skull= os.path.join(bone_registration_path, d.split(os.sep)[-1] + '_' + m.split(os.sep)[-1] + '_' + image_format)
        output_head= os.path.join(bone_registration_path, d.split(os.sep)[-1] + '_' + h.split(os.sep)[-1] + '_' + image_format)
        if save_registered_images:
            output_registered= os.path.join(bone_registration_path, d.split(os.sep)[-1] + '_' + i.split(os.sep)[-1] + '_' + image_format)
        else:
            output_registered= None
        if (not os.path.isfile(output_skull) or not os.path.isfile(output_head) or 
            (save_registered_images and not os.path.isfile(output_registered))):
            maweight.register_and_transform(i, d, [m, h], [output_skull, output_head], moving_mask= b, fixed_mask= db,
                                            registered_image_path= output_registered, threads= threads, params= elastix_params, work_dir= tmp_path, verbose= 0)

  0%|          | 0/170 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Segmentation by Registration after registration with bone

In [5]:
if registration_after_bone_registration:
    for d in tqdm.tqdm(dissected_images):
        for (i, h, m) in zip(manually_segmented_images, manually_segmented_head, manually_segmented_skulls):
            output_skull= os.path.join(output_path, d.split(os.sep)[-1] + '_' + m.split(os.sep)[-1] + '_' + image_format)
            output_head= os.path.join(output_path, d.split(os.sep)[-1] + '_' + h.split(os.sep)[-1] + '_' + image_format)
            if save_registered_images:
                output_registered= os.path.join(output_path, d.split(os.sep)[-1] + '_' + i.split(os.sep)[-1] + '_' + image_format)
            else:
                output_registered= None
            if (not os.path.isfile(output_skull) or not os.path.isfile(output_head) or 
                (save_registered_images and not os.path.isfile(output_registered))):
                maweight.register_and_transform(i, d, [m, h], [output_skull, output_head], 
                                                registered_image_path= output_registered, threads= threads, params= elastix_params, work_dir= tmp_path, verbose= 0)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 179/179 [15:51:24<00:00, 318.91s/it]
